# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770850451265                   -0.53    9.0         
  2   -2.772144071933       -2.89       -1.31    1.0    129ms
  3   -2.772170279855       -4.58       -2.57    1.0    137ms
  4   -2.772170688053       -6.39       -3.65    1.0    163ms
  5   -2.772170722199       -7.47       -4.07    2.0    152ms
  6   -2.772170722989       -9.10       -5.47    1.0    158ms
  7   -2.772170723013      -10.62       -5.39    3.0    172ms
  8   -2.772170723015      -11.73       -6.25    1.0    141ms
  9   -2.772170723015      -13.14       -7.19    2.0    172ms
 10   -2.772170723015   +  -14.65       -7.26    1.0    152ms
 11   -2.772170723015      -13.95       -8.42    1.0    164ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770816491722                   -0.52    9.0         
  2   -2.7

1.7735578528954374

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770779441866                   -0.52    9.0         
  2   -2.772061062468       -2.89       -1.32    1.0    129ms
  3   -2.772083040359       -4.66       -2.44    1.0    160ms
  4   -2.772083339088       -6.52       -3.13    1.0    132ms
  5   -2.772083417472       -7.11       -4.20    2.0    178ms
  6   -2.772083417722       -9.60       -4.67    1.0    169ms
  7   -2.772083417809      -10.06       -5.84    1.0    137ms
  8   -2.772083417811      -11.72       -6.35    2.0    159ms
  9   -2.772083417811      -13.44       -7.00    2.0    192ms
 10   -2.772083417811      -13.64       -8.04    1.0    149ms

Polarizability via ForwardDiff:       1.7725349457631505
Polarizability via finite difference: 1.7735578528954374
